<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Arima Function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'> ARIMA stands for Autoregressive Integrated Moving Average. It is a statistical method used for time series forecasting and analysis. ARIMA is a form of regression analysis that gauges the strength of one dependent variable relative to other changing variables. ARIMA models are popular in various fields, including finance, economics, and environmental science, for predicting future points in a time series based on its historical values.</p>
 
<p style = 'font-size:16px;font-family:Arial'>The following procedure is an example of how to use Arima function:</p>
<li style = 'font-size:16px;font-family:Arial'>Run the ArimaEstimate() function to get the coefficients for the ARIMA model.</li>
<li style = 'font-size:16px;font-family:Arial'>[Optional] Run ArimaValidate() function to validate the 'goodness of fit' of the ARIMA model, when "fit_percentage" argument value is not 100 in ArimaEstimate() function.</li>
<li style = 'font-size:16px;font-family:Arial'>Run the ArimaForecast() function with input from step 1 or step 2 to forecast the future periods beyond the last observed period.</li>
</p>
 
<p style = 'font-size:16px;font-family:Arial'></p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDSeries,
    ArimaEstimate,
    ArimaValidate,
    ArimaForecast,
    Figure,
    plot,
    db_drop_view,
    remove_context,
    copy_to_sql,
    db_drop_table
    )

from teradatasqlalchemy.types import *

from teradataml import to_numeric
# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_Arima.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_SalesForecasting_local');"       # Takes 70 seconds

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial'>2. Preparing Dataset</b>


<p style = 'font-size:16px;font-family:Arial'>
    <li style = 'font-size:16px;font-family:Arial'>Weekly_Sales is our variable of interest. </li>
    <li style = 'font-size:16px;font-family:Arial'>Type, Size, Temperature, isHoliday, Fuel_Price, MarkDown1, MarkDown2, MarkDown3, MarkDown4, MarkDown4 are exogenous variables.</li>
</p>

<p style = 'font-size:16px;font-family:Arial'>We prepare the dataset by creating a view by joining data from Weekly Sales, Stores and features. The view is created using SQL to reduce the number of steps to join and data preocessing which gets used in  further steps.</p>

In [ ]:
query2='''REPLACE VIEW Weekly_Sales_Details AS
SELECT
    w.Sales_date AS times,
    CAST('2012-02-03' AS DATE) AS cutoff_date,
    w.Dept,
    w.Store,
    CAST(w.Sales_Date AS TIMESTAMP) AS Sales_Date,
    ZEROIFNULL(Weekly_Sales) AS Weekly_Sales,
    ZEROIFNULL(Store_Size) AS Store_Size,
    Store_Type AS Store_Type,
    w.IsHoliday,
    ZEROIFNULL(Temperature) AS Temperature,
    ZEROIFNULL(MarkDown1) AS MarkDown1,
    ZEROIFNULL(MarkDown2) AS MarkDown2,
    ZEROIFNULL(MarkDown3) AS MarkDown3,
    ZEROIFNULL(MarkDown4) AS MarkDown4,
    ZEROIFNULL(MarkDown5) AS MarkDown5,
    ZEROIFNULL(CPI) AS CPI,
    ZEROIFNULL(Unemployment) AS Unemployment,
    ZEROIFNULL(Fuel_Price) AS Fuel_Price,
    CAST(TRIM(w.Dept) || TRIM(w.Store) AS INT) AS idcols
FROM
    Demo_SalesForecasting.Weekly_Sales w
LEFT JOIN
    Demo_SalesForecasting.Stores s ON w.Store = s.Store
LEFT JOIN
    Demo_SalesForecasting.Features f ON w.Store = f.store AND w.Sales_Date = f.Sales_Date
WHERE
    w.Store IN (20, 4);
'''

execute_sql(query2)
modeldf=DataFrame.from_query('select * from Weekly_Sales_Details;')

In [ ]:
dfacheck = modeldf.groupby(["idcols"])
dfacheck=dfacheck.count().select(["idcols","count_Sales_Date"])

dfa4=modeldf.join(dfacheck, on = 'idcols', how = "left", lsuffix = 't1', rsuffix = 't2').drop(['idcols_t2'],axis=1)
dfa4=dfa4.assign(idcols = dfa4['idcols_t1'])
dfa4=dfa4.drop(['idcols_t1'],axis=1)

# filter out incomplete time series 

modeldf1 = dfa4[dfa4.count_Sales_Date == 143]
modeldf1.shape

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial'>3. ArimaEstimate</b>
<p style = 'font-size:16px;font-family:Arial'>The ArimaEstimate() function estimates the coefficients corresponding to an ARIMA (AutoRegressive Integrated Moving Average) model, and to fit a series with an existing ARIMA model. The function can also provide the "goodness of fit" and the residuals of the fitting operation. The function generates model layer used as input for the ArimaValidate() and ArimaForecast() functions. This function is for univariate series.</p>

<p style = 'font-size:16px;font-family:Arial'>First we will use Resample function to transform the irregular series into a regular series. The first step is to convert the time series data into TDSeries, which is required for the input time series which are passed to the Resample function.</p>


<p style = 'font-size:16px;font-family:Arial'>The Resample() function transforms an irregular time series into a regular time series. It can also be used to alter the sampling interval for a time series. The Resample functions requires a series as inuput for which we use the TDSeries function.</p>

<p style = 'font-size:16px;font-family:Arial'>TDSeries object from a teradataml DataFrame representing a SERIES in time series which is used as input to Unbounded Array Framework, time series functions. A series is a one-dimensional array. They are the basic input of UAF functions. A series is identified by its series ID, i.e., "id" argument, and indexed by "row_index" argument. Series is passed to and returned from UAF functions as wavelets. Wavelets are collections of rows, grouped by one or more fields, and ordered on the "row_index" argument.</p>

In [ ]:
from teradataml import TDSeries, Resample

data_series_df = TDSeries(data=modeldf1,
                              id="idcols",
                              row_index=("Sales_Date"),
                              row_index_style= "TIMECODE",
                              payload_field="Weekly_Sales",
                              payload_content="REAL")

uaf_out1 = Resample(data=data_series_df,
                        interpolate='LINEAR',
                        timecode_start_value="TIMESTAMP '2010-02-05 00:00:00'",
                        timecode_duration="WEEKS(1)")

df=uaf_out1.result
df1=df.select(['idcols','ROW_I', 'Weekly_Sales']).assign(Sales_Date=df.ROW_I)
df1

<p style = 'font-size:16px;font-family:Arial'>Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(ArimaEstimate)

<p style = 'font-size:16px;font-family:Arial'>The ArimaEstimate() function takes one or two inputs, the second input is optional. The first input is a time series. The second input references the model context. When only one input is passed in, the ArimaEstimate() function operates in a coefficient estimate mode. When two inputs are passed in, ArimaEstimate() function operates in a model apply mode. When the second input is passed in, user must include an input format mode argument.</p>

In [ ]:
from teradataml import ArimaEstimate
# Execute ArimaEstimate function.
data_series_df_1 = TDSeries(data=df1,
                              id="Sales_Date",
                              row_index=("idcols"),
                              row_index_style= "SEQUENCE",
                              payload_field="Weekly_Sales",
                              payload_content="REAL")

arima_est_out = ArimaEstimate(data1=data_series_df_1,
                            nonseasonal_model_order=[2,1,1],
                            constant=False,
                            algorithm="MLE",
                            coeff_stats=True,
                            fit_metrics=True,
                            residuals=True,
                            fit_percentage=80)

In [ ]:
est_result=arima_est_out.fitresiduals
est_result = est_result.groupby('Sales_Date').avg()
est_result

<p style = 'font-size:16px;font-family:Inter'>We plot the Actual Value of Weekly Sales vs the Calculated Value of the ArimaEstimate function. </p>

In [ ]:
from teradataml import Figure
figure = Figure(width=800, height=500, heading="Comparison of Actual vs Predicted Sales")
est_result.plot(x=est_result.Sales_Date, y=[est_result.avg_ACTUAL_VALUE, est_result.avg_CALC_VALUE], 
             style=['dark orange', 'green'], xlabel='Sales Date', ylabel='Sales',  grid_color='black',xtick_format='YYYY-MM',
                   grid_linewidth=0.5, grid_linestyle="-", legend=['Actual Value','Predicted Value'],figure=figure)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial'>4. ArimaValidate</b>
<p style = 'font-size:16px;font-family:Arial'>The ArimaValidate() function performs an in-sample forecast for both seasonal and non-seasonal auto-regressive (AR), moving-average (MA), ARIMA models and Box-Jenkins seasonal ARIMA model formula followed by an analysis of the produced residuals. The aim is to provide a collection of metrics useful to select the model and expose the produced residuals such that multiple model validation and statistical tests can be conducted.</p>      


<p style = 'font-size:16px;font-family:Arial'>Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(ArimaValidate)

In [ ]:
from teradataml import TDAnalyticResult, ArimaValidate
data_art_df = TDAnalyticResult(data=arima_est_out.result)


arima_val_out = ArimaValidate(data=data_art_df, fit_metrics=True, residuals=True)

In [ ]:
val_result=arima_val_out.fitresiduals
val_result = val_result.groupby('Sales_Date').avg()
val_result

<p style = 'font-size:16px;font-family:Inter'>We plot the Actual Value of Weekly Sales vs the Calculated Value of the ArimaValidate function. </p>

In [ ]:
from teradataml import Figure
figure = Figure(width=800, height=500, heading="Comparison of Actua vs Predicted")
val_result.plot(x=val_result.Sales_Date, y=[val_result.avg_ACTUAL_VALUE, val_result.avg_CALC_VALUE], 
             style=['dark orange', 'green'], xlabel='Sales Date', ylabel='Sales',  grid_color='black',
                   grid_linewidth=0.5, grid_linestyle="-", legend=['Actual Value','Predicted Value'],figure=figure)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial'>5. ArimaForecast</b>
<p style = 'font-size:16px;font-family:Arial'>The ArimaForecast() function is used to forecast a user-defined number of periods based on models fitted from the ArimaEstimate() function.</p>  

<p style = 'font-size:16px;font-family:Arial'>Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(ArimaForecast)

 

<p style = 'font-size:16px;font-family:Arial'>Here we are considering 7 periods (forecast_periods=7)</p>

In [ ]:
from teradataml import ArimaForecast
arima_estimate_op = ArimaEstimate(data1=data_series_df_1,
                                      nonseasonal_model_order=[2,1,1],
                                      constant=False,
                                      algorithm="MLE",
                                      coeff_stats=True,
                                      fit_metrics=True,
                                      residuals=True,
                                      fit_percentage=100)

# Create teradataml TDAnalyticResult object over the result attribute of 'arima_estimate_op'
data_art_df = TDAnalyticResult(data=arima_estimate_op.result)
 
arima_forcast_out = ArimaForecast(data=data_art_df, forecast_periods=7)

In [ ]:
forecast_result=arima_forcast_out.result
forecast_result = forecast_result.groupby('ROW_I').avg()
forecast_result

<p style = 'font-size:16px;font-family:Inter'>We plot the Forecasted Value of Weekly Sales for the defined number of periods. </p>

In [ ]:
from teradataml import Figure
figure = Figure(width=800, height=500, heading="Forecast Sales")
forecast_result.plot(x=forecast_result.ROW_I, y=forecast_result.avg_FORECAST_VALUE, 
              xlabel='Forecast Period', ylabel='Forecast Sales',  grid_color='black',
                   grid_linewidth=0.5, grid_linestyle="-", figure=figure)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial'>6. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up intermediate tables.</p>

In [ ]:
db_drop_view('Weekly_Sales_Details')

<p style = 'font-size:18px;font-family:Arial'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_SalesForecasting');" 
#Takes 45 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>ArimaEstimate function reference: <a href = 'https://docs.teradata.com/search/all?query=ArimaEstimate&content-lang=en-US'>here</a></li>
    <li>ArimaValidate function reference: <a href = 'https://docs.teradata.com/search/all?query=ArimaValidate&content-lang=en-US'>here</a></li>
    <li>ArimaForecast function reference: <a href = 'https://docs.teradata.com/search/all?query=ArimaForecast&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>